# 네이버 영화 리뷰 데이터 1000개 수집

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [9]:
browser = webdriver.Chrome()
url = "https://movie.naver.com/movie/point/af/list.nhn"
browser.get(url)

In [10]:
from tqdm import tqdm
reviewData = list()
# links = browser.find_element_by_class_name('paging').find_elements_by_tag_name('a')
_iter = 1000
for page in range(_iter):
    for row in browser.find_elements_by_tag_name('tr')[1:]:
        title = row.find_element_by_class_name('title').text.split('\n')[0]
        point = row.find_element_by_class_name('point').text
        text = row.find_element_by_class_name('title').text.split('\n')[1][:-3]
        reviewData.append((title, point, text))
    browser.find_element_by_class_name('pg_next').click()

In [11]:
import csv

csvFile = open("./reviewData.csv", "a", newline="\n")
csvWriter = csv.writer(csvFile)

for row in reviewData:
    csvWriter.writerow(row)

csvFile.close()

# 긍정데이터와 부정데이터에서 많이나온 명사/조사/형용사/감탄사 top5씩 뽑기

In [12]:
from collections import Counter, defaultdict
# counter는 value가 정렬 되어있음
from konlpy.tag import Komoran

ma = Komoran()

In [13]:
ma.tagset

{'EC': '연결 어미',
 'EF': '종결 어미',
 'EP': '선어말어미',
 'ETM': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JKB': '부사격 조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JKV': '호격 조사',
 'JX': '보조사',
 'MAG': '일반 부사',
 'MAJ': '접속 부사',
 'MM': '관형사',
 'NA': '분석불능범주',
 'NF': '명사추정범주',
 'NNB': '의존 명사',
 'NNG': '일반 명사',
 'NNP': '고유 명사',
 'NP': '대명사',
 'NR': '수사',
 'NV': '용언추정범주',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SH': '한자',
 'SL': '외국어',
 'SN': '숫자',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'VA': '형용사',
 'VCN': '부정 지정사',
 'VCP': '긍정 지정사',
 'VV': '동사',
 'VX': '보조 용언',
 'XPN': '체언 접두사',
 'XR': '어근',
 'XSA': '형용사 파생 접미사',
 'XSN': '명사파생 접미사',
 'XSV': '동사 파생 접미사'}

# 최근 인기많은 영화  N개 과연 클러스터링을하면 N개로 군집화가 잘 될지